<a href="https://colab.research.google.com/github/marcelobenedito/quarantine_covid19_behavior_analysis/blob/master/quarantine_covid19_behavior_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Quarantine Covid-19 Behavior Analysis**

*It will be collect data tweets about COVID-19, quarantine and related about. This content will analysed to extract sentiment and main user behavior that makes don't stay home.*

**Install libraries**

In [26]:
!pip3 install unidecode
!pip3 install googletrans
!pip3 install twitterscraper
!pip3 install emoji
!pip3 install simplejson

     |████████████████████████████████| 133kB 2.8MB/s 


**Required imports**

In [104]:
import string
import time
import datetime as dt
import numpy as np
import pandas as pd
import re
import nltk
import simplejson
from unidecode import unidecode
from googletrans import Translator
from twitterscraper import query_tweets
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from emoji import demojize

**Create a funcion to search tweets**

In [3]:
def search_tweets(search_filter, since, until, limit, language):
  return query_tweets(query = search_filter, begindate = since, enddate = until, limit = limit, lang = language)

**Defining filters used in search**

In [36]:
""" não estou saindo "não estou saindo" (quarentena OR covid) (#covid-19 OR #coronavírus OR #coronavirus OR #covid OR #quarentena) lang:pt until:2020-01-31 since:2020-01-01 -filter:replies """

contains_both_words = ''
exact_phrase = ''
contains_any_words = '(quarentena OR covid OR coronavirus OR isolamento OR festa OR role OR evento OR balada OR sair OR saindo)'
contains_any_hashtags = ''
no_retweet = '-filter:replies'
language = 'pt'
since = dt.date(2020,1,1)
until = dt.date(2020,1,2)
limit = None

search_filter = contains_both_words + ' ' + exact_phrase + ' ' + contains_any_words + ' ' + contains_any_hashtags + ' ' + no_retweet

**Extracting tweets based on search filter**

In [37]:
tweets = search_tweets(search_filter, since, until, limit, language)

Failed to parse JSON while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1212523827260993538-1212523890540462082&q=%20%20(quarentena%20OR%20covid%20OR%20coronavirus%20OR%20isolamento%20OR%20festa%20OR%20role%20OR%20evento%20OR%20balada%20OR%20sair%20OR%20saindo)%20%20-filter%3Areplies%20since%3A2020-01-01%20until%3A2020-01-02&l=pt"
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/twitterscraper/query.py", line 107, in query_single_page
    html = json_resp['items_html'] or ''
KeyError: 'items_html'
Failed to parse JSON while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1212523827260993538-1212523890540462082&q=%20%20(quarentena%20OR%20covid%20OR%20coronavirus%20OR%20isolamento%20OR%20festa%20OR%20

**Transform Json to DataFrame and export to CSV file**

In [38]:
df = pd.DataFrame({
    'tweet_id': tweet.tweet_id, 
    'text': tweet.text,  
    'tweet_url': tweet.tweet_url,
    'retweets': tweet.retweets,
    'replies': tweet.replies,
    'is_replied': tweet.is_replied,
    'is_reply_to': tweet.is_reply_to,
    'user_id': tweet.user_id, 
    'created_at': tweet.timestamp
} for tweet in tweets)

df.to_csv('tweets.csv', encoding='utf-8')

**Printing found tweets**

In [39]:
df.head()

,tweet_id,text,tweet_url,retweets,replies,is_replied,is_reply_to,user_id,created_at
0,1212523890540462082,"Esse eu vou dá tchau pra vida de festa, tá bom?",/NetoMiguel02/status/1212523890540462082,0,0,False,False,1190617786294374400,2020-01-01 23:59:57
1,1212523890439786496,o quanto as coisas demoram p sair da minha cab...,/amandiiiix/status/1212523890439786496,0,0,False,False,1064533534109589504,2020-01-01 23:59:57
2,1212523879970885633,meu momento pós role sempre é baseado em pensa...,/inouesz/status/1212523879970885633,0,0,False,False,1046604175717601280,2020-01-01 23:59:55
3,1212523878477635584,Deu janeiro e eu quero sair do emprego eai kkk...,/Haile_Din/status/1212523878477635584,0,0,False,False,1032060705221083137,2020-01-01 23:59:54
4,1212523878230171648,o que vc diria?\n\n1- cuida dela pq ela é espe...,/Laura_Liiotta/status/1212523878230171648,0,0,False,False,723473031293743104,2020-01-01 23:59:54


**Open stored tweets from CSV file**

In [100]:
# file = open('tweets.csv', encoding='utf-8').read()
df = pd.read_csv('tweets.csv')


**Probably will need to make a translate from Portuguese to English**

In [101]:
tweets = df['text']
total = len(tweets)
translated_quantity = 0
english_tweets = []

print('{} [INFO] Starting tweet translation...'.format(dt.datetime.now()))
for tweet in tweets:  
  english_tweets.append(Translator().translate(unidecode(tweet)).text)
  translated_quantity += 1
  print('\r{} [INFO] {} translated tweets of {} from Portuguese to English.'.format(dt.datetime.now(), translated_quantity, total), end='')

#df['text'].apply([lambda tweet: ' '.join(Translator().translate(unidecode(text)).]text for text in tweet))
df.insert(3, 'english_text', english_tweets)

print('\n{} [INFO] Tweet translation successfully!'.format(dt.datetime.now()))

df.head()

2020-08-01 16:59:06.949221 [INFO] Starting tweet translation...
2020-08-01 16:59:10.237234 [INFO] 20 translated tweets of 20 from Portuguese to English.
2020-08-01 16:59:10.238878 [INFO] Tweet translation successfully!


,Unnamed: 0,tweet_id,text,english_text,tweet_url,retweets,replies,is_replied,is_reply_to,user_id,created_at
0,0,1212523890540462082,"Esse eu vou dá tchau pra vida de festa, tá bom?","This one I go bye to party life, okay?",/NetoMiguel02/status/1212523890540462082,0,0,False,False,1190617786294374400,2020-01-01 23:59:57
1,1,1212523890439786496,o quanto as coisas demoram p sair da minha cab...,how long things take to get out of my head is ...,/amandiiiix/status/1212523890439786496,0,0,False,False,1064533534109589504,2020-01-01 23:59:57
2,2,1212523879970885633,meu momento pós role sempre é baseado em pensa...,my moment can always be based on thinking abou...,/inouesz/status/1212523879970885633,0,0,False,False,1046604175717601280,2020-01-01 23:59:55
3,3,1212523878477635584,Deu janeiro e eu quero sair do emprego eai kkk...,It was January and I want to quit my job kkkkk...,/Haile_Din/status/1212523878477635584,0,0,False,False,1032060705221083137,2020-01-01 23:59:54
4,4,1212523878230171648,o que vc diria?\n\n1- cuida dela pq ela é espe...,what would you say?\n\n1- take care of her bec...,/Laura_Liiotta/status/1212523878230171648,0,0,False,False,723473031293743104,2020-01-01 23:59:54


**Data preprocessing**

This process is used to preprocess the tweet text:

 - Tokenize words;
 - Remove all stop words; 
 - Punctuaction rules; 
 - Unused characters;
 - Links from tweets.

In [105]:
# Converting to lowercase
tweets = df.english_text.str.lower()

# Removing punctuation rules
tweets = tweets.str.translate(str.maketrans('', '', string.punctuation))

# Removing unused links
tweets = tweets.str.replace(r"(http|@)\S+", "")

# Remove special chars
tweets = tweets.apply(demojize)
tweets = tweets.str.replace(r"::", ": :")
tweets = tweets.str.replace(r"’", "'")
tweets = tweets.str.replace(r"[^a-z\':_]", " ")

# Remove repetitions
pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
tweets = tweets.str.replace(pattern, r"\1")

# Transform short negation form
tweets = tweets.str.replace(r"(can't|cannot)", 'can not')
tweets = tweets.str.replace(r"n't", ' not')

# Spliting text into words
# tweets = word_tokenize(tweets, 'english')

# Removing stop words
nltk.download('stopwords')
stopwords = stopwords.words('english')
stopwords.remove('not')
stopwords.remove('nor')
stopwords.remove('no')

tweets.apply(
    lambda tweet: ' '.join([word for word in tweet.split() if word not in stopwords])
)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0                            one go bye party life okay
1                long things take get head unbelievable
2     moment always based thinking story im going te...
3                               january want quit job k
4     would say take care special stick ass ask dscl...
5                     mark role chic best goes straight
6     last role isas house pictwittercom epxqizxqo p...
7     dont want call snap fingers anything go otaria...
8        month birthday already planning surprise party
9         go parties playlist agr party hot minus songs
10    afraid judged leaving room la girl eats sleeps...
11                      many pictures coming shit great
12    found best party game ever check pictwittercom...
13                                        yes bore role
14    c knows partners birthday deg january candy no...
15    year cried bus cried classroom cried riding bi...
16                           hair beautiful ill even go
17                                 want know fir